In [5]:
import pandas as pd
import numpy as np
import requests

In [44]:
df = pd.read_csv('../raw_data/air_pollution_data.csv')
df.head()

,who_region,iso3,country_name,city,year,version,pm10_concentration,pm25_concentration,no2_concentration,pm10_tempcov,pm25_tempcov,no2_tempcov,type_of_stations,reference,web_link,population,population_source,latitude,longitude,who_ms
0,4_Eur,ESP,Spain,A Coruna/ESP,2013.0,"V4.0 (2018), V4.0 (2018), V4.0 (2018), V4.0 (2...",23.238,11.491,28.841,87.0,46.0,93.0,"Urban, Urban, Suburban",NaN,NaN,246146.0,"manual, manual, manual, manual",43.367900,-8.418571,1
1,4_Eur,ESP,Spain,A Coruna/ESP,2014.0,"V6.0 (2023), V6.0 (2023), V6.0 (2023)",27.476,15.878,19.575,96.0,88.0,95.0,"Urban, Urban, Suburban",NaN,NaN,247604.0,NaN,43.368033,-8.418233,1
2,4_Eur,ESP,Spain,A Coruna/ESP,2015.0,"V6.0 (2023), V6.0 (2023), V6.0 (2023), V6.0...",25.515,14.004,22.731,98.0,71.0,98.0,"Urban, Urban, Suburban, Suburban",NaN,NaN,247604.0,NaN,43.370375,-8.422900,1
3,4_Eur,ESP,Spain,A Coruna/ESP,2016.0,"V6.0 (2023), V6.0 (2023), V6.0 (2023), V6.0...",23.057,13.160,20.204,98.0,98.0,98.0,"Urban, Urban, Suburban, Suburban",NaN,NaN,247604.0,NaN,43.370375,-8.422900,1
4,4_Eur,ESP,Spain,A Coruna/ESP,2017.0,"V6.0 (2023), V6.0 (2023), V6.0 (2023), V6.0...",26.849,14.114,21.543,97.0,97.0,98.0,"Urban, Urban, Suburban, Suburban",NaN,NaN,247604.0,NaN,43.370375,-8.422900,1


In [46]:
df["population"].isna().sum()

17665

In [8]:
cities = df['city'].unique()

In [9]:
citydf=[]
countries=[]
for city in cities: 
    
    parts= city.split("/")
    citydf.append(parts[0])
    countries.append(parts[1])

In [12]:
data['city']['population']

242619

In [18]:
print(f"{city}: {population}")

Wagrowiec: 55381


In [140]:
##"DIfferent API "

##for city in citydf:
##response = requests.get(f"https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/geonames-all-cities-with-a-population-500/records?select=name%2C%20population")

In [13]:
# Creating a dictionary to store city names and their populations

city_populations = {}  

for city in citydf:
    response = requests.get(f"https://api.algobook.info/v1/city?search={city}")
    data = response.json()
    if data and 'city' in data and data['city'] is not None:
        population = data['city']['population']
    else:
        population = "NaN"
    city_populations[city] = population
    print(f"{city}: {population}")  # Print each city's population

print(city_populations)


A Coruna: 242619
A La Shan Zuo Qi: NaN
Aachen: 265208
Aadorf: 7620
Aalborg: NaN
Aalen: 67085
Aalesund: NaN
Aamjiwnaang: NaN
Aarau: 15501
Aarhus: NaN
Aarschot: 27656
Aba: 2757729
Abadan: 370180
Abanto Y Ciervana: NaN
Abasolo: 25821


KeyboardInterrupt: 

In [142]:
import json

filename = 'population_dictionary.json'
with open(filename, 'w') as f:
    json.dump(city_populations, f)

print(f"Dictionary exported as {filename}")

Dictionary exported as population_dictionary.json


In [14]:
def populate_column_with_dict(df, json_file):
    # Load the dictionary from the json file
    with open(json_file, 'r') as f:
        city_dict = json.load(f)

    df['populationAPI'] = df['cities_name'].map(city_dict)

    return df

In [53]:
cities = df["city"]

cities_name = []
countries_name = []

for city in cities: 
    parts = city.split("/")
    if len(parts) == 2:
        cities_name.append(parts[0])
        countries_name.append(parts[1])

df['cities_name'] = cities_name
df['countries_name'] = countries_name

In [54]:
df = populate_column_with_dict(df, "population_dictionary.json")

In [55]:
df['populationAPI']

0        242619
1        242619
2        242619
3        242619
4        242619
          ...  
40093       NaN
40094       NaN
40095       NaN
40096       NaN
40097       NaN
Name: populationAPI, Length: 40098, dtype: object

In [56]:
(df["populationAPI"].value_counts()/40098)*100

populationAPI
NaN       37.577934
0          0.765624
158094     0.054866
28783      0.052372
276964     0.052372
            ...    
14277      0.002494
11112      0.002494
7185       0.002494
812799     0.002494
78364      0.002494
Name: count, Length: 3869, dtype: float64

In [57]:
import pandas as pd
import numpy as np

# Create a new column 'merged_pop' with all conditions specified
df['merged_pop'] = np.where(~df['population'].isna(), df['population'],
                            np.where(df['population'].isna() & ~df['populationAPI'].isna(), df['populationAPI'],
                                     np.nan))

print(df[['population', 'populationAPI', 'merged_pop']])


       population populationAPI merged_pop
0        246146.0        242619   246146.0
1        247604.0        242619   247604.0
2        247604.0        242619   247604.0
3        247604.0        242619   247604.0
4        247604.0        242619   247604.0
...           ...           ...        ...
40093         NaN           NaN        NaN
40094         NaN           NaN        NaN
40095         NaN           NaN        NaN
40096         NaN           NaN        NaN
40097         NaN           NaN        NaN

[40098 rows x 3 columns]


In [59]:
df["merged_pop"].value_counts()

merged_pop
NaN          6630
0             210
2264823.0      54
23822.0        33
5919.0         31
             ... 
69905.0         1
71840.0         1
3729.0          1
19909.0         1
32132           1
Name: count, Length: 8098, dtype: int64